In [1]:
import os
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama

In [2]:
def open_file(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            contents = file.read()
        return contents
    except FileNotFoundError:
        print("File not found:")
    except Exception as e:
        print("Error:")


In [7]:
client = Ollama(model="llama3:8b")

chunk_size = 1000
chunk_overlap = 0.25

file_path = "documents/Hardware.txt"
persist_directory = "chroma"

embedding = OllamaEmbeddings(model="bge-m3")

criar_db = False

In [ ]:
if criar_db:
    print("Criando Banco Vetorial")

    # Abrindo o arquio
    texto = open_file(file_path)
    filename = os.path.basename(file_path)
    metadatas = [{"nome do arquivo": filename}]

    text_splitter = CharacterTextSplitter(separator="\n", chunk_size=chunk_size,
                                        chunk_overlap=int(chunk_size * chunk_overlap),
                                        length_function=len,
                                        is_separator_regex=False,
                                        )


    all_splits = text_splitter.create_documents([texto], metadatas=metadatas)

    vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)
    

else:
    print("Carregando Banco Vetorial")
    try:
        vectorstore = Chroma(embedding_function=embedding, persist_directory=persist_directory)
        print("Banco de vetores carregado com sucesso.")
    except Exception as e:
        print(f"Erro ao carregar banco de vetores: {e}")

    
question = "Qual a diferença entre Unidade de Controle e Unidade Lógica Aritmética?"

docs = vectorstore.similarity_search_with_score(question, k=4)

Carregando Banco Vetorial
Banco de vetores carregado com sucesso.


In [9]:
def enviar_pergunta(pergunta, contexto):
    try:
        prompt = f"""
        Pergunta: {pergunta}

        Utilize as informações a seguir para responder a pergunta:
        {contexto}


        Resposta:
        """
        resposta = client.invoke(prompt)
        return resposta

    except Exception as e:
        return f"Ocorreu um erro: {e}"
    
    
resposta = enviar_pergunta(question, str(docs))

print("Resposta:", resposta)

Resposta: A pergunta é sobre a diferença entre Unidade de Controle e Unidade Lógica Aritmética.

Segundo as informações fornecidas, a Unidade de Controle (CU) é um componente essencial da Unidade Central de Processamento (CPU) que direciona as operações do processador. Ele busca instruções da memória, decodifica-as e gera os sinais de controle necessários para executar as instruções.

Já a Unidade Lógica Aritmética (ALU) é a parte do processador que realiza operações aritméticas e lógicas. Ela executa operações como adição, subtração, multiplicação e divisão, além de operações lógicas como AND, OR e NOT.

Em resumo, a Unidade de Controle é responsável por gerenciar as instruções do processador e controlar a execução das operações, enquanto a Unidade Lógica Aritmética é responsável por realizar as operações aritméticas e lógicas propriamente ditas.

Portanto, a principal diferença entre as duas unidades está na função que cada uma desempenha no processador: a Unidade de Controle se preo